In [ ]:
import os
import pybedtools
import pandas as pd

In [ ]:
sp = "Fly"
gv = "dm6"

In [ ]:
all_peaks = pd.read_csv("../data/PeakFiles/%sTFPeaksPrimaryTargets.tsv" % sp, sep="\t")
genome_fasta = "/home/jg2447/slayman/genome/%s.fa" % gv
chr2len = pd.read_csv(genome_fasta+".fai", sep="\t", header=None, index_col=0)[1].to_dict()
cluster_info = pd.read_csv("../data/PeakFiles/%sMetaPeaksPrimaryTargets.tsv" % sp, sep="\t")
cluster2size = cluster_info.set_index("metaPeak")["numPeaks"].to_dict()

In [ ]:
for suffix in ["AllPeaks", "Top20Percent", "Top20PercentRandom_Rep1", "Top20PercentRandom_Rep2", "Top20PercentRandom_Rep3", "RemoveHOT", "RemoveHOTRandom_Rep1", "RemoveHOTRandom_Rep2", "RemoveHOTRandom_Rep3", "RemoveUltraHOT", "RemoveUltraHOTRandom_Rep1", "RemoveUltraHOTRandom_Rep2", "RemoveUltraHOTRandom_Rep3"]:
    os.makedirs("../result/motif/%sBed_%s" % (sp, suffix), exist_ok=True)
    for peak_name, peak_df in all_peaks.groupby("experiment"):
        peak_name = peak_name.replace("(", "").replace(")", "")
        peak_df["chr_max"] = peak_df["chrom"].map(chr2len)
        peak_df.loc[peak_df["chromEnd"] >= peak_df["chr_max"], "chromEnd"] = peak_df.loc[peak_df["chromEnd"] >= peak_df["chr_max"], "chr_max"].values
        if suffix == "AllPeaks":
            peak_df = peak_df.sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]

        elif suffix == "Top20Percent":
            npeaks = int(peak_df.shape[0] * 0.2)
            peak_df = peak_df.sort_values("signalValue", ascending=False).iloc[:npeaks, [0,1,2]]
        elif suffix.startswith("Top20PercentRandom"):
            npeaks = int(peak_df.shape[0] * 0.2)
            rs = int(suffix.split("_Rep")[1])
            peak_df = peak_df.sample(npeaks, random_state=rs)
            peak_df = peak_df.sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]

        elif suffix == "RemoveHOT":
            peak_df["metaPeak_size"] = peak_df["metaPeak"].map(cluster2size)
            if sp == "Fly":
                peak_df = peak_df[(peak_df["metaPeak_size"] < 277)].sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]
            elif sp == "Worm":
                peak_df = peak_df[(peak_df["metaPeak_size"] < 85)].sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]
        elif suffix == "RemoveUltraHOT":
            peak_df["metaPeak_size"] = peak_df["metaPeak"].map(cluster2size)
            if sp == "Fly":
                peak_df = peak_df[(peak_df["metaPeak_size"] < 53)].sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]
            elif sp == "Worm":
                peak_df = peak_df[(peak_df["metaPeak_size"] < 31)].sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]

        elif suffix.startswith("RemoveHOTRandom"):
            peak_df["metaPeak_size"] = peak_df["metaPeak"].map(cluster2size)
            if sp == "Fly":
                sub_n = peak_df[(peak_df["metaPeak_size"] < 277)].shape[0]  
            elif sp == "Worm":
                sub_n = peak_df[(peak_df["metaPeak_size"] < 85)].shape[0]
            rs = int(suffix.split("_Rep")[1])
            peak_df = peak_df.sample(sub_n, random_state=rs)
            peak_df = peak_df.sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]

        elif suffix.startswith("RemoveUltraHOTRandom"):
            peak_df["metaPeak_size"] = peak_df["metaPeak"].map(cluster2size)
            if sp == "Fly":
                sub_n = peak_df[(peak_df["metaPeak_size"] < 53)].shape[0]  
            elif sp == "Worm":
                sub_n = peak_df[(peak_df["metaPeak_size"] < 31)].shape[0]
            rs = int(suffix.split("_Rep")[1])
            peak_df = peak_df.sample(sub_n, random_state=rs)
            peak_df = peak_df.sort_values("signalValue", ascending=False).iloc[:, [0,1,2]]

        if peak_df.shape[0] > 2:
            peak_df.to_csv("../result/motif/%sBed_%s/%s.bed" % (sp, suffix, peak_name), sep="\t", header=False, index=False)

    os.makedirs("../result/motif/%sFasta_%s/" % (sp, suffix), exist_ok=True)
    files = [ii.replace(".bed", "") for ii in os.listdir("../result/motif/%sBed_%s/" % (sp, suffix)) if ii.endswith(".bed")]
    for ff in files:
        bed = pybedtools.BedTool("../result/motif/%sBed_%s/%s.bed" % (sp, suffix, ff))
        bed.sequence(fi=genome_fasta).save_seqs("../result/motif/%sFasta_%s/%s.fasta" % (sp, suffix, ff))

    os.makedirs("../result/motif/%sSTREME_%s/" % (sp, suffix), exist_ok=True)
    files = [ii.replace(".fasta", "") for ii in os.listdir("../result/motif/%sFasta_%s/" % (sp, suffix)) if ii.endswith(".fasta")]
    with open("./joblist_%s_%s" % (sp, suffix), "w") as f:
        for ff in files:
            f.write("module load MEME; streme --p ../result/motif/%sFasta_%s/%s.fasta --oc ../result/motif/%sSTREME_%s/%s/ --verbosity 1\n" % (sp, suffix, ff, sp, suffix, ff))